Identify duplicates by image mean
does not work :(

In [90]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import xarray as xr
import re
import rtree
import shapely
import pickle

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

In [151]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset

tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
tiles_xml_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_complete_dataset_path, exist_ok=True)

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)

correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips = pd.read_csv("correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips.csv")

In [ ]:
## identify correct images by mean

In [299]:
correct_xs = []
correct_ys = []
correct_chip_names = []
correct_chip_means = []

for tile_in_dir in tqdm.tqdm(os.listdir(tiles_complete_dataset_path)):
    tile = cv2.imread(os.path.join(tiles_complete_dataset_path, tile_in_dir))
    file_name, ext = os.path.splitext(tile_in_dir) # File name
        
    #get tile shape
    item_dim = int(512)          
    tile_height,  tile_width,  tile_channels = tile.shape #the size of the tile #determine tile dimensions
    row_index = math.ceil(tile_height/512) #divide the tile into 512 by 512 chips (rounding up)
    col_index = math.ceil(tile_width/512)
    #print(row_index, col_index)
    count = 1  
    for y in range(0, col_index):
        for x in range(0, row_index):
            correct_chip_name = file_name+ '_' + str(count).zfill(6) + '.jpg'
            correct_chip_names.append(correct_chip_name)
            correct_xs.append(x)
            correct_ys.append(y)
            
            #create a numpy array of each correctly chipped images 
            correct_image = fc.tile_to_chip_array(tile, x, y, item_dim)
            correct_chip_means.append(np.mean(correct_image))
            
            count += 1
            if (np.mean(correct_image) == 0) and (np.sum(correct_image) != 0):
                fig = plt.figure()
                plt.imshow(correct_image)
                print(x,y)                            
                plt.title(file_name+ '_' + str(count).zfill(6) + '.jpg')
                plt.show() 
correct_chip_means = np.array(correct_chip_means)

100%|██████████| 1922/1922 [4:31:47<00:00,  8.48s/it]  


In [321]:
###read in incorrect images
incorrect_labeled_chip_names_by_subfolder = pd.read_csv("incorrect_labeled_chip_names_by_subfolder.csv")
indicies_of_incorrect_labeled_chip_image_remaining  = pd.read_csv("indicies_of_incorrect_labeled_chip_image_remaining.csv")
indicies_of_incorrect_labeled_chip_image_remaining = indicies_of_incorrect_labeled_chip_image_remaining["indicies_of_incorrect_labeled_chip_image_remaining"].tolist()

incorrect_labeled_chip_image_array_remaining = np.zeros((len(incorrect_labeled_chip_names_by_subfolder),512,512, 3), dtype='uint8')
incorrect_labeled_chip_means_remaining = np.empty((len(incorrect_labeled_chip_names_by_subfolder)))

for num in tqdm.tqdm(indicies_of_incorrect_labeled_chip_image_remaining):    
    incorrect_chip_array_remaining = cv2.imread(incorrect_labeled_chip_names_by_subfolder["incorrect_chip_paths"][num])  #open image
    incorrect_labeled_chip_image_array_remaining[num,:,:,:] = incorrect_chip_array_remaining
    incorrect_labeled_chip_means_remaining[num] = np.mean(incorrect_chip_array_remaining)

    percent_errors = []
   
    for correct_chip_mean in correct_chip_means:
        dividend = np.absolute(correct_chip_means - incorrect_labeled_chip_means_remaining[num])
        divisor = correct_chip_means + np.finfo(float).eps
        percent_errors = (np.divide(dividend, divisor) * 100)
        print(np.min(percent_errors),np.max(percent_errors))        
    percent_errors = []

  0%|          | 0/801 [01:18<?, ?it/s]


KeyboardInterrupt: 

In [311]:
#confirm that np.mean is a reliable metric for assessing the similarity between images
#images in storage space have been compressed/chipped previously (are known to be correct)
incorrect_labeled_image_names = []
percent_errors = []
correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips_index = []

for num in tqdm.tqdm(indicies_of_incorrect_labeled_chip_image_remaining):
    corrected_incorrect_image = cv.imread(correct_incorrect_image)
    corrected_incorrect_image_mean = np.mean(corrected_incorrect_image)
    corrected_incorrect_image_means.append(corrected_incorrect_image_mean)
    
    correct_incorrect_standard_image_name = incorrect_labeled_chip_names_by_subfolder["incorrect_chip_names"][num].split("_",4)[-1]
    correct_incorrect_standard_image_names.append(correct_incorrect_standard_image_name)
    
    index = np.where(np.array(correct_chip_names) == correct_incorrect_standard_image_name)[0][0]
    correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips_index.append(index)
    percent_errors.append( abs((correct_chip_means[index] - corrected_incorrect_image_mean)/correct_chip_means[index]) * 100 )

 30%|███       | 241/801 [01:22<03:23,  2.75it/s]C:\Users\RAPIDU~1\AppData\Local\Temp/ipykernel_9240/691543178.py:17: RuntimeWarning: divide by zero encountered in double_scalars
  percent_errors.append( abs((correct_chip_means[index] - corrected_incorrect_image_mean)/correct_chip_means[index]) * 100 )
100%|██████████| 801/801 [04:32<00:00,  2.94it/s]


In [294]:
#confirm that np.mean is a reliable metric for assessing the similarity between images
#images in storage space have been compressed/chipped previously (are known to be correct)
corrected_incorrect_image_means = []
correct_incorrect_standard_image_names = []
percent_errors = []
correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips_index = []

for correct_incorrect_image in tqdm.tqdm(correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips["correct_image_paths"]):
    corrected_incorrect_image = cv.imread(correct_incorrect_image)
    corrected_incorrect_image_mean = np.mean(corrected_incorrect_image)
    corrected_incorrect_image_means.append(corrected_incorrect_image_mean)
    
    correct_incorrect_image_name = correct_incorrect_image.split("\\")[-1]
    correct_incorrect_standard_image_name = correct_incorrect_image_name.split("_",4)[-1]
    correct_incorrect_standard_image_names.append(correct_incorrect_standard_image_name)
    
    index = np.where(np.array(correct_chip_names) == correct_incorrect_standard_image_name)[0][0]
    correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips_index.append(index)
    percent_errors.append( abs((correct_chip_means[index] - corrected_incorrect_image_mean)/correct_chip_means[index]) * 100 )

100%|██████████| 549/549 [03:21<00:00,  2.72it/s]


In [298]:
#Percent error is not a reliable method to identify similar images
"""
for idx, percent_error in enumerate(percent_errors):
    if percent_error > 0.05:
        print(idx, percent_error)
        print(correct_incorrect_standard_image_names[idx])
        #print(correct_incorrect_standard_image_names[idx])
        #fig, (ax1, ax2) = plt.subplots(1, 2)
        #ax1.set_title('correct_image')
        #ax1.imshow(fc.tile_to_chip_array(cv.imread(os.path.join(tiles_complete_dataset_path,"m_3008824_ne_16_060_20191115.tif")), 20, 13, 512))
        #ax2.set_title('labeled_chip_array')
        #ax2.imshow(cv.imread([string for string in correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips["correct_image_paths"] if correct_chip_names[index] in string][0]))
        #plt.show() 
"""

57 0.07699870757074227
m_3008135_se_17_060_20191119_000380.jpg
85 0.0659614601407446
m_2708205_sw_17_060_20191129_000195.jpg
89 0.05522360157848654
m_2708205_nw_17_060_20191129_000495.jpg
90 0.051763068265803076
m_2708205_nw_17_060_20191129_000520.jpg
91 0.06130410518456201
m_2608064_nw_17_060_20191120_000265.jpg
101 0.08530277334966058
m_2909002_nw_15_060_20190707_000055.jpg
126 0.0650390624601024
m_2608064_nw_17_060_20191120_000289.jpg
141 0.08193209432087858
m_2708205_nw_17_060_20191129_000470.jpg
143 0.0647004741674657
m_2608064_nw_17_060_20191120_000361.jpg
158 0.06817002894536185
m_3008845_nw_16_060_20181204_000320.jpg
173 0.05917165288773231
m_3009058_sw_15_060_20190831_000269.jpg
186 0.06409701335989597
m_3009058_sw_15_060_20190831_000451.jpg
207 0.06438676200696698
m_3009058_sw_15_060_20190831_000372.jpg
215 0.059979154485006916
m_3008845_nw_16_060_20181204_000271.jpg
234 0.06106161090156692
m_3812264_sw_10_060_20180715_000254.jpg
248 0.2890173185302233
m_3907508_nw_18_060_201

In [ ]:
#for num in tqdm.tqdm(np.sort(indicies_of_incorrect_labeled_chip_image_remaining)):    
    #fig = plt.figure()
    #plt.imshow(cv.imread(incorrect_labeled_chip_names_by_subfolder["incorrect_chip_paths"][num]))
    #print(incorrect_labeled_chip_names_by_subfolder["incorrect_chip_paths"][num])
    #plt.show() 
    
index = np.where(np.array(percent_errors) == np.sort(percent_errors)[10])[0][0]
fc.tile_to_chip_array(tile, x, y, item_dim)
fig = plt.figure()
plt.imshow(cv.imread(incorrect_labeled_chip_names_by_subfolder["incorrect_chip_paths"][num]))
print(incorrect_labeled_chip_names_by_subfolder["incorrect_chip_paths"][num])
plt.show() 

fig, (ax1,ax2) = plt.subplots(figsize=(20, 20))
    
plt.imshow(cv.imread(os.path.join(tiles_complete_dataset_path,"m_4007424_nw_18_060_20190830.tif")))#, correct_xs[index], correct_ys[index], 512))
#ax2.set_title('labeled_chip_array')
#ax2.imshow(cv.imread(incorrect_labeled_chip_names_by_subfolder["incorrect_chip_paths"][num]))
plt.show() 

#sorted(correct_chip_means)
#sortecorrect_chip_names[index]

index = np.where(np.array(percent_errors) == 0.2890173185302233)[0][0]
correct_incorrect_standard_image_names[index]

In [223]:
correct_chip_means[index]
#corrected_incorrect_image_means[index]
np.mean(cv.imread('//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Jackson/student_reviewed_images3/chips_positive\\pa_60cm_2019_39075_m_3907508_nw_18_060_20191015_000240.jpg'))

81.86643600463867

In [220]:
np.mean(fc.tile_to_chip_array(cv.imread(os.path.join(tiles_complete_dataset_path,"m_3907508_nw_18_060_20191015.tif")), 14, 9, 512))

81.63050969441731

In [ ]:
correct_chip_name = correct_chip_names[0]
            incorrect_image_indicies = [ind for ind, string in enumerate(correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips["correct_image_paths"]) if correct_chip_name in string]
            if len(incorrect_images_indicies) > 0:
                for incorrect_image_index in incorrect_image_indicies:
                    index = np.where(np.array(percent_errors) == 0.2890173185302233)[0][0]
                    correct_incorrect_standard_image_names[index]
                    incorrect_image = cv.imread(correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips["correct_image_paths"][incorrect_image_index])
                    corrected_incorrect_image_mean = np.mean(incorrect_image)
                    corrected_incorrect_image_means[incorrect_image_index] = corrected_incorrect_image_mean

In [485]:
correct_images_for_incorrect_labeled_chip_names_identified_from_confirmed_correct_positive_chips["correct_image_paths"][1]

'//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//unverified_images\\student_reviewed_unverified_images_set1/Alex/student_reviewed_images1/chips_positive\\fl_60cm_2019_30081_m_3008135_se_17_060_20191119_000355.jpg'